In [5]:
!pip install pinecone pymongo transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.6/587.6 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.0/240.0 kB 20.8 MB/s eta 0:00:00


In [6]:
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi

uri = "mongodb+srv://<username>:<password>@cluster0.9cktzei.mongodb.net/?retryWrites=true&w=majority&appName=Cluster0"

# Create a new client and connect to the server
client = MongoClient(uri, server_api=ServerApi('1'))

# Send a ping to confirm a successful connection
try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)

Pinged your deployment. You successfully connected to MongoDB!


In [10]:
PINECONE_API_KEY = "API KEY"

A Pinecone index is like a specialized database that stores vectors (numerical embeddings) and lets you search through them efficiently.

In a RAG pipeline:

You convert chunks of text into embeddings.

You store them in this index.

You later query the index to get similar/relevant chunks based on user input.

In [11]:
from pinecone import Pinecone

pc = Pinecone(api_key=PINECONE_API_KEY)
index = pc.Index("mongo")

In [12]:
index

Imagine MongoDB as a digital library:

client → The whole library building

db = client["mytestdb"] → A room (database) named "mytestdb"

collection = db["mytestcollection"] → A bookshelf (collection) inside that room

Documents (data) → Books you store or retrieve from that shelf

In [13]:
db=client["mytestdb"]

In [15]:
collection=db["mytestcollection"]

In [16]:
!pip install sentence_transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 122.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 83.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 103.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitl

In [19]:
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# Goal: 🔁 Keep Pinecone (vector DB) updated in real-time with changes happening in a MongoDB collection.
# If you insert, update, or delete a document in MongoDB, Pinecone will react accordingly.

# open up change stream cursor
cursor = collection.watch(full_document='updateLookup')
print("Change stream is now open.")
while True:
    change = next(cursor)
    # If a new document is inserted into the collection, replicate its vector in Pinecone
    if change['operationType'] == 'insert':
      document = change['fullDocument']
      # convert the document's name into an embedding
      vector = embedding_model.encode(document['fullplot'])
      # Ensure the vector is a flat list of floats (and possibly convert to float64)
      vector = vector.tolist()  # Convert from numpy array to list
      vector = [float(x) for x in vector]  # Convert elements to float (usually float64)
      # Prepare the data for Pinecone upsert, which requires a tuple of (id, vector)
      # Assuming 'document['_id']' is the unique ID for the upsert operation
      upsert_data = (str(document['_id']), vector)
      # Insert into Pinecone
      index.upsert([upsert_data])  # Note that upsert_data is enclosed in a list

    elif change['operationType'] == 'update':
      document = change['fullDocument']
      document_id = document['_id']
      updated_fields = change['updateDescription']['updatedFields']

      # if the change is in the name field, generate the embedding and insert
      if updated_fields.get('fullplot'):
        vector = embedding_model.encode(updated_fields['fullplot'])
        upsert_data = (str(document_id), vector)
        # Insert into Pinecone
        index.upsert([upsert_data])  # Note that upsert_data is enclosed in a list

        #pinecone.upsert(index_name="myindex", data=vector, ids=[str(document_id)])

    # If a document is deleted from the collection, remove its vector from Pinecone
    elif change['operationType'] == 'delete':
      index.delete(ids=[str(change['documentKey']['_id'])])

Change stream is now open.
